In [2]:
import pandas as pd
import datetime
import glob

In [9]:
columns = ['Ticker', 'Date/Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']

dtypes = {
    'Ticker': 'string',
    'Open': 'float64',
    'High': 'float64',
    'Low': 'float64',
    'Close': 'float64',
    'Volume': 'float64',
    'Open Interest': 'float64'
}

def dateparse(d,t):
    dt = d + " " + t
    return datetime.datetime.strptime(dt, '%Y/%m/%d %H:%M')

def dateParseSpot(d,t):
    dt = d + " " + t
    return datetime.datetime.strptime(dt, '%m/%d/%Y %H:%M:%S')

# Prepare spot data parquet

In [12]:
finalDf = pd.DataFrame(columns=columns)

dateparse = lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S')

for file in glob.glob('C:/Personal/Projects/PyAlgoMate/pyalgomate/backtesting/data/data/indices/*.csv'):
    df = pd.read_csv(file,
                     skiprows=1,
                     header=None,
                     names=['Ticker', 'Date', 'Time', 'Open', 'High',
                            'Low', 'Close', 'Volume', 'Open Interest'],
                     dtype=dtypes,
                     parse_dates={'Date/Time': ['Date', 'Time']}, date_parser=dateparse)

    df.Ticker = df.Ticker.str.replace('.NSEBANK',
                                      'BANKNIFTY').replace('.CNX100',
                                                           'CNX100').replace('.CNXIT',
                                                                             'CNXIT').replace('.NSEI',
                                                                                              'NIFTY')

    df = df[columns]

    finalDf = pd.concat([finalDf, df],
                        ignore_index=True).sort_values(['Ticker',
                                                        'Date/Time']).drop_duplicates(subset=['Ticker',
                                                                                              'Date/Time'],
                                                                                      keep='first')

finalDf.to_parquet('C:/Personal/Projects/PyAlgoMate/pyalgomate/backtesting/data/2023/spot-apr23.parquet',
                   index=False)


C:\Users\ng664891\AppData\Local\Temp\ipykernel_46696\1141976948.py:6: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(file,
C:\Users\ng664891\AppData\Local\Temp\ipykernel_46696\1141976948.py:6: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(file,
C:\Users\ng664891\AppData\Local\Temp\ipykernel_46696\1141976948.py:6: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(file,
C:\Users\ng664891\AppData\Local\Temp\ipykernel_46696\1141976948.py:6: FutureWarning: The argument 'date_parser' is dep

# Prepare expiry data

In [75]:
expiry = '20APR23'
weekly = True
directory = 'C:/Personal/Projects/PyAlgoMate/pyalgomate/backtesting/data/data/Nifty Options/Expiry 20th April/*.csv'

# define regex patterns to match and replace
pattern1 = r"^([a-zA-Z]+)WK(\d+)(P|C)E$"
pattern2 = r"^([a-zA-Z]+)(\d+)(P|C)E$"
replace1 = r"\1Expiry\3\2"

finalDf = pd.DataFrame(columns=columns)
dateparse = lambda x: datetime.datetime.strptime(x, '%Y/%m/%d %H:%M')

for file in glob.glob(directory):
    df = pd.read_csv(file,
                     header=None,
                     names=['Ticker', 'Date', 'Time', 'Open', 'High',
                            'Low', 'Close', 'Volume', 'Open Interest'],
                     dtype=dtypes,
                     parse_dates={'Date/Time': ['Date', 'Time']}, date_parser=dateparse)

    df['Ticker'] = df['Ticker'].str.replace(
        pattern1 if weekly == True else pattern2, replace1, regex=True).str.replace('Expiry', expiry)

    df = df[columns]

    finalDf = pd.concat([finalDf, df],
                        ignore_index=True).sort_values(['Ticker',
                                                        'Date/Time']).drop_duplicates(subset=['Ticker',
                                                                                              'Date/Time'],
                                                                                      keep='first')

finalDf.to_parquet(f'C:/Personal/Projects/PyAlgoMate/pyalgomate/backtesting/data/2023/nifty-{expiry}.parquet',
                   index=False)


C:\Users\ng664891\AppData\Local\Temp\ipykernel_46696\598006669.py:14: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(file,
C:\Users\ng664891\AppData\Local\Temp\ipykernel_46696\598006669.py:14: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(file,
C:\Users\ng664891\AppData\Local\Temp\ipykernel_46696\598006669.py:14: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(file,
C:\Users\ng664891\AppData\Local\Temp\ipykernel_46696\598006669.py:14: FutureWarning: The argument 'date_parser' is dep

In [76]:
parquetFile = 'C:/Personal/Projects/PyAlgoMate/pyalgomate/backtesting/data/2023/nifty-06APR23.parquet'

In [77]:
parquetDf = pd.read_parquet(parquetFile)

In [78]:
parquetDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315602 entries, 0 to 315601
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Ticker         315602 non-null  string        
 1   Date/Time      315602 non-null  datetime64[ns]
 2   Open           315602 non-null  float64       
 3   High           315602 non-null  float64       
 4   Low            315602 non-null  float64       
 5   Close          315602 non-null  float64       
 6   Volume         315602 non-null  float64       
 7   Open Interest  315602 non-null  float64       
dtypes: datetime64[ns](1), float64(6), string(1)
memory usage: 19.3 MB


In [79]:
parquetDf.tail()

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest
315597,NIFTY06APR23P19100,2023-04-03 11:17:00,1780.00,1780.00,1780.00,1780.00,50.0,300.0
315598,NIFTY06APR23P19100,2023-04-03 15:25:00,1734.60,1734.60,1734.60,1734.60,100.0,100.0
315599,NIFTY06APR23P19200,2023-04-03 10:49:00,1817.15,1817.15,1817.15,1817.15,50.0,50.0
315600,NIFTY06APR23P19200,2023-04-03 11:01:00,1881.00,1881.00,1881.00,1881.00,50.0,100.0
315601,NIFTY06APR23P19200,2023-04-06 11:05:00,1600.00,1600.00,1600.00,1600.00,50.0,50.0


# Combine Weekly and Monthly files into one file

In [80]:
finalDf = pd.DataFrame(columns=columns)

for file in glob.glob('C:/Personal/Projects/PyAlgoMate/pyalgomate/backtesting/data/2023/nifty*.parquet'):
    finalDf = pd.concat([finalDf, pd.read_parquet(file)],
                        ignore_index=True).sort_values(['Ticker',
                                                        'Date/Time']).drop_duplicates(subset=['Ticker',
                                                                                              'Date/Time'],
                                                                                      keep='first')

finalDf.sort_values(['Ticker',
                     'Date/Time']).drop_duplicates(subset=['Ticker',
                                                           'Date/Time'],
                                                   keep='first').to_parquet(
    'C:/Personal/Projects/PyAlgoMate/pyalgomate/backtesting/data/2023/nifty/04.parquet', index=False)
